In [1]:
import pandas as pd

In [2]:
hwis_cali = pd.read_csv("../data/local_data/monthly/HWs_california_NOAA.csv")
hwis_chile = pd.read_csv("../data/local_data/monthly/HWs_chile_central.csv")

In [3]:
hwis_cali["Date"] = pd.to_datetime(hwis_cali["Date"],format='%Y-%m')
hwis_cali.set_index('Date', inplace=True)
hwis_cali = hwis_cali[hwis_cali.index.year >= 1972]
hwis_cali

,HWN,HWF,HWD,HWM,HWA
Date,,,,,
1972-01-01,0.1,0.3,0.3,0.071875,0.145303
1972-02-01,0.8,3.5,2.7,1.041480,2.121939
1972-03-01,1.7,12.5,8.6,3.591644,7.481389
1972-04-01,0.0,0.0,0.0,0.000000,0.000000
1972-05-01,0.9,3.1,2.4,1.720682,2.671300
...,...,...,...,...,...
2023-08-01,0.3,1.1,1.1,0.573921,1.020132
2023-09-01,0.1,0.3,0.3,0.104056,0.182232
2023-10-01,1.0,4.5,3.5,2.140890,3.330383


In [4]:
hwis_chile["Date"] = pd.to_datetime(hwis_chile["Date"],format='%Y-%m')
hwis_chile.set_index('Date', inplace=True)
hwis_chile = hwis_chile[hwis_chile.index >= "1971-12"]
hwis_chile

,HWN,HWF,HWD,HWM,HWA
Date,,,,,
1971-12-01,0.0,0.0,0.0,0.000000,0.000000
1972-01-01,0.1,0.3,0.3,0.451728,0.981709
1972-02-01,0.1,0.3,0.3,0.129245,0.199264
1972-03-01,0.0,0.0,0.0,0.000000,0.000000
1972-04-01,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...
2023-08-01,0.1,0.3,0.3,0.349259,0.520451
2023-09-01,0.0,0.0,0.0,0.000000,0.000000
2023-10-01,0.1,0.3,0.3,0.040896,0.055833


In [5]:
def monthly_to_seasonly(original, name, seasons):
    df = original.copy()  # Optional: work on a copy

    # Extract month and year for grouping
    df['month'] = df.index.month
    df['year'] = df.index.year
    if 12 in seasons:
        df.loc[df['month'] == 12, 'year'] += 1  # Shift December to next year's summer}

    # Keep only summer months
    summer_df = df[df['month'].isin(seasons)]

    summer_agg = summer_df.groupby('year').agg({
        'HWN': 'sum',
        'HWF': 'sum',
        'HWD': 'max',
        'HWM': 'mean',
        'HWA': 'max'
    })
    summer_agg.index = summer_agg.index.astype(str) + "-07"
    summer_agg.index.name = 'Date'
    summer_agg.to_csv(f"../data/local_data/summerly/HWs_{name}.csv")

In [6]:
statlist_cali = ['USC00040983', 'USC00041194', 'USC00041253', 'USC00040741', 'USC00040212', 'USC00040931', 'USC00041277', 'USC00041244', 'USC00040693', 'USC00040449']
statlist_chile = [330007, 330020, 330030, 340031, 360011, 380013, 360019, 390006, 400009, 410005]

for stat in statlist_cali:
    df_stat = pd.read_csv(f"../data/local_data/monthly/HWs_{stat}.csv")
    df_stat["Date"] = pd.to_datetime(df_stat["Date"],format='%Y-%m')
    df_stat.set_index('Date', inplace=True)
    df_stat = df_stat[df_stat.index >= "1971-12"]
    monthly_to_seasonly(df_stat, stat, [6, 7, 8])

# for stat in statlist_chile:
#     df_stat = pd.read_csv(f"../data/local_data/monthly/HWs_{stat}.csv")
#     df_stat["Date"] = pd.to_datetime(df_stat["Date"],format='%Y-%m')
#     df_stat.set_index('Date', inplace=True)
#     df_stat = df_stat[df_stat.index >= "1971-12"]
#     monthly_to_seasonly(df_stat, stat, [12, 1, 2])

## Spatial Average

In [7]:
data_cali = pd.read_csv(f"../data/local_data/summerly/HWs_USC00040983.csv")

data_chile = pd.read_csv(f"../data/local_data/summerly/HWs_330007.csv")
indices = ["HWN", "HWF", "HWD", "HWM", "HWA"]


for stat in statlist_chile[1:]:
    df = pd.read_csv(f"../data/local_data/summerly/HWs_{stat}.csv", usecols=indices)
    data_chile = pd.concat([data_chile, df], axis=1)
for stat in statlist_cali[1:]:
    df = pd.read_csv(f"../data/local_data/summerly/HWs_{stat}.csv", usecols=indices)
    data_cali = pd.concat([data_cali, df], axis=1)
data_chile.set_index("Date", inplace=True)
data_cali.set_index("Date", inplace=True)

In [8]:

df_mean_chile = pd.DataFrame(columns=indices, index=data_chile.index)
df_mean_cali = pd.DataFrame(columns=indices, index=data_cali.index)

for index in indices:
    df_mean_chile[index] = data_chile[index].mean(axis=1)
    df_mean_cali[index] = data_cali[index].mean(axis=1)

## rmbr i removed 2024-01 (created cs of december 2023)
save = True
if save:
    # df_mean_chile.to_csv("../data/local_data/summerly/HWs_chile_central.csv")
    df_mean_cali.to_csv("../data/local_data/summerly/HWs_california.csv")